In [17]:
import numpy as np
import pandas as pd


In [7]:
df=pd.read_csv('chatbot_dataset.csv')
df.head(2)

,User ID,User Utterance,Bot Response,Timestamp,Context/Session ID,Entities,User Feedback,Conversation Outcome,User Profile,Channel/Platform,Language,User Emotion/Sentiment,Location,User Segment
0,uKqYhMMQ5S,Charge bar between follow student.,Important law into large example range. Player...,2023-10-31 18:02:06,4dfe56dc-efe2-49c6-be9f-ce5b84ca4de4,event,negative,incomplete,Annette Henderson,social media,German,confused,Sydney,returning customers
1,YOonrpgxp9,Bad every reflect huge contain.,Policy argue agree character go recent. When r...,2023-10-31 18:02:06,74c97616-9df6-460d-991d-c6f6f6ae5354,location,neutral,specific outcome,Nicholas Haney,mobile app,Chinese,frustrated,London,returning customers


In [11]:
x=df['User Utterance']
y=df['Bot Response']

In [15]:
y[0]

'Important law into large example range. Player seem force with partner sometimes happen southern.'

In [16]:
x[0]

'Charge bar between follow student.'

In [18]:
vocab=["My name is Bablu","Bablu goes to college"]

In [19]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(vocab)

In [20]:
sequences=tokenizer.texts_to_sequences(vocab)

In [21]:
sequences

[[2, 3, 4, 1], [1, 5, 6, 7]]

In [22]:
tokenizer.word_index

{'bablu': 1, 'my': 2, 'name': 3, 'is': 4, 'goes': 5, 'to': 6, 'college': 7}

In [38]:
# 4 dimensions for each word
import math
def positionalEncodings(sequence):
    
    PE=[]
    d_model=4
    for pos in range(sequence):
        EncodingsOfEach=[]
        for i in range(d_model):
            if i%2==0:
                temp=pos/(math.pow(10000,(2*i)/d_model))
                EncodingsOfEach.append(math.sin(temp))
            else:
                temp=pos/(math.pow(10000,(2*(i-1))/d_model))
                EncodingsOfEach.append(math.cos(temp))
        PE.append(EncodingsOfEach)
    return PE

In [46]:
tokens = ["The", "cat", "sat"]
PE=(positionalEncodings(len(tokens)))


In [52]:
a=[1,0,1,0]
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result

array([1, 0, 1, 0])

In [70]:
a=np.array([1,0,1,0])
b=np.array([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
result=np.matmul(a,b)
result
print(a.shape)
print(b.shape)
print(result.shape)


(4,)
(4, 4)
(4,)


In [ ]:
#input will be the embedding(+PE) 
# eg:[[1,0,1,0],[0,1,0,1],[1,1,1,1]]
def selfAttention(embeddings,W_q, W_k,W_v):
    
